In [ ]:
!pip install -q pymupdf sentence-transformers faiss-cpu groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 16.9 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from groq import Groq


In [ ]:
PDF_PATH = "/content/sample-service-manual.pdf"  # change if needed

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

raw_text = extract_text_from_pdf(PDF_PATH)
print("PDF loaded successfully")


PDF loaded successfully


In [ ]:
def chunk_text(text, chunk_size=1200, overlap=200):
    chunks = []
    start = 0
    text_length = len(text)

    while start < text_length:
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap

    return chunks

chunks = chunk_text(raw_text)
print(f"Total chunks created: {len(chunks)}")


Total chunks created: 863


In [ ]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [ ]:
embeddings = embedding_model.encode(chunks, show_progress_bar=True)
embeddings = np.array(embeddings).astype("float32")

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

print("FAISS index created")


Batches:   0%|          | 0/27 [00:00<?, ?it/s]

FAISS index created


In [ ]:
def retrieve_relevant_chunks(query, top_k=10):
    query_embedding = embedding_model.encode([query]).astype("float32")
    distances, indices = index.search(query_embedding, top_k)
    return [chunks[i] for i in indices[0]]


In [ ]:
client = Groq(api_key="")  # 🔑 put your API key here


In [ ]:
def extract_vehicle_specs(context, query):
    prompt = f"""
You are a vehicle service manual assistant.

Context:
{context}

Question:
{query}

Rules:
- Answer ONLY using the provided context
- Do NOT use prior knowledge or external information
- If a value is not explicitly present in the context, do NOT include it in the output
- Do NOT guess, infer, assume, or estimate
- For Torque,there are 3 possible unit values.at max 2 unit values are available in pdf. extract them only if exist
- Do NOT invent part numbers, fluids, materials, or specifications
- If query is expecting textual answer like reasons, actions etc, return in textual explanation format
- if value is - , tell value not mentioned in pdf
- If no valid answer exists in the context, return an empty JSON array []
- If a specification is given in multiple units, include all units as separate JSON objects
- Do NOT mix or convert units
- Use the exact JSON schema provided
- Use consistent component naming and spec_type values
- Return only the minimum number of JSON objects required
- Ignore procedural text, notes, and unrelated specifications
- Return ONLY valid JSON

format for textual explanation:
  [
  {{
    "Answer": ""
  }}
]


format for ranges of values:
[
  {{
    "component": "",
    "spec_type": "",
    "Range": "",
    "unit": ""
  }}
]


Format for TORQUE and Capacities and for other metrics:
[
  {{
    "component": "",
    "spec_type": "",
    "value": "",
    "unit": ""
  }}
]

format for part number or any non-metric items(which have no units) :
[
  {{
    "component": "",
    "spec_type": "",
    "value": ""
  }}
]






"""

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You extract structured vehicle specifications."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )

    return response.choices[0].message.content


In [ ]:
query = "what is the torque of Wheel bearing and wheel hub bolts ?"
retrieved_chunks = retrieve_relevant_chunks(query)

context = " ".join(retrieved_chunks)[:6000]

output = extract_vehicle_specs(context, query)
print(output)



[
  {
    "component": "Wheel bearing and wheel hub bolts",
    "spec_type": "Torque",
    "value": "175",
    "unit": "Nm"
  },
  {
    "component": "Wheel bearing and wheel hub bolts",
    "spec_type": "Torque",
    "value": "129",
    "unit": "lb-ft"
  }
]
